#  Notes

1) Resources to solve binary integer programming problem in Python.
    
* https://towardsdatascience.com/integer-programming-in-python-1cbdfa240df2
    
2) Want to solve Sudoku using binary integer programming with following model
    
$$\min \sum_{i=1}^{729} x_i$$ doesn't really matter, just as formality
        
$$\text{subject to } \sum_{k = 1}^{9} x_{81 \cdot i + 9 \cdot j + k} = 1 \text{for } i = 0 \ldots 8,j = 0 \ldots 8$$ -> each cell can only have 1 number
        
$$\sum_{j = 0}^{8} x_{81 \cdot i + 9 \cdot j + k} = 1 \text{for } k = 1 \ldots 9,i = 0 \ldots 8$$ -> each number can only appear once in a row
        
$$\sum_{i = 0}^{8} x_{81 \cdot i + 9 \cdot j + k} = 1 \text{for } j = 0 \ldots 8,k = 1 \ldots 9$$ -> each number can only appear once in a column
        
$$\sum_{i \in I_m, j \in J_n} x_{81 \cdot i + 9 \cdot j + k} = 1 \text{for } I_m = \{3m+1,3m+2,3m+3\}, J_n = \{3n+1,3n+2,3n+3\}, m = 0 \ldots 2, n = 0 \ldots 2, k = 1 \ldots 9 $$ -> each number can only appear once in a 3 x 3 box

$$x_{81 \cdot i + 9 \cdot j + k} = 1 \text{for } cell_{(i,j)} = k $$


The next thing to do is to represent it in matrix form, from a 3D matrix with entries $m_{i,j,k}$ to $x_{81(i-1)+9(j-1)+k}$

The objective function can be written as 

$$\begin{bmatrix} 1 & \ldots & 1 \end{bmatrix} \begin{bmatrix} x_1 \\ \vdots \\ x_{729} \end{bmatrix}$$

First constraint matrix can be written as an $81 \times 729$ matrix

$$\begin{bmatrix} 1 & \ldots & 1 & 0 & \ldots & \ldots & \ldots & \ldots & 0 \\
 0 & \ldots & 0 & 1 & \ldots & 1 & 0 & \ldots & 0 \\
 \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
 0 & \ldots & \ldots & \ldots & \ldots & 0 & 1 & \ldots & 1\end{bmatrix}$$

Similarly, 2nd to 4th constraint matrices can each be written as $81 \times 729$ matrix 

Overall matrix is $324 \times 729$ matrix, plus additional rows 


# Algorithm

1) start by inputting the sudoku problem

* can be done by reading in .txt file 

* if no .txt file, can key in the values for it to generate .txt file

2) based on .txt file, put in the "additional rows" of constraints, add it to the $324 \times 729$ matrix, call this matrix `constraint`

3) we also need a matrix $b$ such that $Ax=b$, hence $b$ is a vectors of 1's with the same number of rows in `constraint`

4) run the ILP and obtain the solution vector $x$

5) use double for loops to extract the values 

    for i = 0 to 8
        for j = 0 to 8
            new x = x[81i+9j+1 ... 81i+9j+9]
            idx = index of x == 1
            print (idx)
        end
        print newline
    end

In [ ]:
0 9 18 81 90 99 162 171 180 
27 36 45 108 117 126 189 198 207
54 63 72 135 144 153 216 225 234
243 252 261 324 333 342 405 414 423

# Making an app

### Goal: To make an app that starts out with an empty sudoku grid with 9x9 cells, let user enter the puzzle, select Solve, and obtain the solution, then click Restart option to start again

Resources:

* http://newcoder.io/gui/part-2/

* https://realpython.com/mobile-app-kivy-python/